## 1.0 通过SSH登录设备
通过SSH 登录设备，并通过ssh 获取设备的一些信息。

这次使用的lib 为 [netmiko]。(https://github.com/ktbyers/netmiko)

这是一个开源的python 第三方库。目前支持大量的网络设备。其就是操作网络设备而开发的第三方库。

## 1.1 登录SSH后，获取设备的接口信息

In [1]:
from netmiko import ConnectHandler

In [2]:
iosxr3 = {
    'device_type': 'cisco_xr',
    'ip':   '172.20.3.102',
    'username': 'admin',
    'password': 'admin',
    'port' : 22          # optional, defaults to 22    
}

net_connect = ConnectHandler(**iosxr3)

output = net_connect.send_command('show ip int brief')
print(output)


Thu Aug  9 23:59:21.209 UTC

Interface                      IP-Address      Status          Protocol Vrf-Name
MgmtEth0/0/CPU0/0              172.20.3.102    Up              Up       default 
GigabitEthernet0/0/0/0         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/1         unassigned      Up              Up       default 
GigabitEthernet0/0/0/2         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/3         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/4         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/5         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/6         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/7         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/8         unassigned      Shutdown        Down     default 
GigabitEthernet0/0/0/9         unassigned      Shutdown        Down     default

## 1.2 修改接口的IP地址信息

In [3]:
interface_cfg = ["interface GigabitEthernet0/0/0/1",
                "no shutdown",
                "commit",
                "end"]

output = net_connect.send_config_set(interface_cfg)
print(output)

config term

Thu Aug  9 05:13:33.476 UTC
RP/0/0/CPU0:xrv2(config)#interface GigabitEthernet0/0/0/1

RP/0/0/CPU0:xrv2(config-if)#no shutdown

RP/0/0/CPU0:xrv2(config-if)#commit

Thu Aug  9 05:13:33.936 UTC
RP/0/0/CPU0:xrv2(config-if)#end

RP/0/0/CPU0:xrv2#
